# SDK demo

### Preparation

Install Azure ML SDK

In [3]:
!pip install azure-ai-ml

     |████████████████████████████████| 4.0 MB 6.3 MB/s eta 0:00:01
     |████████████████████████████████| 238 kB 86.9 MB/s eta 0:00:01
     |████████████████████████████████| 49 kB 3.2 MB/s  eta 0:00:01
     |████████████████████████████████| 84 kB 1.0 MB/s  eta 0:00:01
     |████████████████████████████████| 252 kB 82.3 MB/s eta 0:00:01
     |████████████████████████████████| 130 kB 53.5 MB/s eta 0:00:01
     |████████████████████████████████| 409 kB 63.3 MB/s eta 0:00:01
  Created wheel for strictyaml: filename=strictyaml-1.6.2-py3-none-any.whl size=123923 sha256=60f51677d99c8457e9bf1c34ce768d9adf8506793eb45347c5725b8cecd162bb
  Stored in directory: /home/azureuser/.cache/pip/wheels/e9/0b/fc/5beda6bad2ff803e820e157845679794a18f83e442da1e9f4d
Successfully built strictyaml
ERROR: tensorflow 2.2.1 has requirement h5py<2.11.0,>=2.10.0, but you'll have h5py 3.7.0 which is incompatible.
ERROR: tensorflow 2.2.1 has requirement numpy<1.19.0,>=1.16.0, but you'll have numpy 1.21.6 which is i

In [6]:
# Import required libraries
from azure.ai.ml import MLClient

Authenticate

In [4]:
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential

try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    # This will open a browser page for
    credential = InteractiveBrowserCredential()

In [7]:
try:
    ml_client = MLClient.from_config(credential=credential)
except Exception as ex:
    # NOTE: Update following workspace information if not correctly configure before
    client_config = {
        "subscription_id": "<SUBSCRIPTION_ID>",
        "resource_group": "<RESOURCE_GROUP>",
        "workspace_name": "<AML_WORKSPACE_NAME>",
    }

    if client_config["subscription_id"].startswith("<"):
        print(
            "please update your <SUBSCRIPTION_ID> <RESOURCE_GROUP> <AML_WORKSPACE_NAME> in notebook cell"
        )
        raise ex
    else:  # write and reload from config file
        import json, os

        config_path = "../.azureml/config.json"
        os.makedirs(os.path.dirname(config_path), exist_ok=True)
        with open(config_path, "w") as fo:
            fo.write(json.dumps(client_config))
        ml_client = MLClient.from_config(credential=credential, path=config_path)
print(ml_client)

Found the config file in: /config.json


### Create Data from file

In [8]:
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes

my_data = Data(
    path="https://tkubicastore.blob.core.windows.net/datasets/creditcard.csv",
    type=AssetTypes.URI_FILE,
    description="Example dataset for credit card fraud detection",
    name="creditcard_sdk"
)

ml_client.data.create_or_update(my_data)

Data({'skip_validation': False, 'mltable_schema_url': None, 'referenced_uris': None, 'type': 'uri_file', 'is_anonymous': False, 'auto_increment_version': False, 'name': 'creditcard_sdk', 'description': 'Example dataset for credit card fraud detection', 'tags': {}, 'properties': {}, 'id': '/subscriptions/d3b7888f-c26e-4961-a976-ff9d5b31dfd3/resourceGroups/data-demo-azureml/providers/Microsoft.MachineLearningServices/workspaces/itckftsmsixv/data/creditcard_sdk/versions/1', 'Resource__source_path': None, 'base_path': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/itckftsmsixv-instance/code/dataplayground/azurerml/sdk_demo', 'creation_context': <azure.ai.ml.entities._system_data.SystemData object at 0x7f48612b2c70>, 'serialize': <msrest.serialization.Serializer object at 0x7f4863d26640>, 'version': '1', 'latest_version': None, 'path': 'https://tkubicastore.blob.core.windows.net/datasets/creditcard.csv', 'datastore': None})

### Run simple job to print this data

In [11]:
from azure.ai.ml import command
from azure.ai.ml.entities import Data
from azure.ai.ml import Input
from azure.ai.ml.constants import AssetTypes

# === Note on path ===
# can be can be a local path or a cloud path. AzureML supports https://`, `abfss://`, `wasbs://` and `azureml://` URIs.
# Local paths are automatically uploaded to the default datastore in the cloud.
# More details on supported paths: https://docs.microsoft.com/azure/machine-learning/how-to-read-write-data-v2#supported-paths

inputs = {
    "input_data": Input(type=AssetTypes.URI_FILE, path="azureml:creditcard_sdk:1")
}

job = command(
    code="../src", 
    command="python head_data.py --input_data ${{inputs.input_data}}",
    inputs=inputs,
    environment="AzureML-sklearn-0.24-ubuntu18.04-py37-cpu:46",
    compute="itckftsmsixv-instance",
)

# submit the command
returned_job = ml_client.jobs.create_or_update(job)
# get a URL for the status of the job
returned_job.services["Studio"].endpoint

'https://ml.azure.com/runs/frosty_stone_brl7cxjqpc?wsid=/subscriptions/d3b7888f-c26e-4961-a976-ff9d5b31dfd3/resourcegroups/data-demo-azureml/workspaces/itckftsmsixv&tid=d6af5f85-2a50-4370-b4b5-9b9a55bcb0dc'

Go to Job run and check its output